# How to select an objective function using information theory

 [![GitHub tag (latest by date)](https://img.shields.io/github/v/tag/hytest-org/workflow-hodson-2022-objective-benchmark)](https://github.com)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hytest-org/workflow-hodson-2022-objective-benchmark/blob/main/01-objective-benchmark-demo.ipynb)

## Abstract
Science tests competing theories or models by evaluating the similarity of their predictions against observational experience. 
Thus, how we measure similarity fundamentally determines what we learn.
In machine learning and scientific modeling, similarity metrics are used as objective functions.
A classic example being mean squared error, which is the optimal measure of similarity when errors are normally distributed and independent and identically distributed (iid). 
In many cases, however, the error distribution is neither normal nor iid, so it is left to the scientist to determine an appropriate objective.
Here, we review how information theory can guide that selection, then demonstrate the approach with a simple hydrologic model.

## Introduction
Science seeks to create useful representations of reality in the form of hypotheses, models, or theories.
What sets science apart from other pursuits is that it rigorously tests those representations against observational experience,
favoring those that best fit the evidence.
An analogous process occurs when calibrating a numerical model or evaluating among competing models.
To select the ``best'' model, experiment by varying the model while keeping the test data and objective fixed.
If mean squared error (MSE) is the objective, compute the MSE between the test data and the model predictions, then select the model with the lowest MSE.
But why choose MSE and not another objective function?
The answer: MSE is the optimal measure when errors are normally distributed and iid.
But for many problems, the true error distribution is complex or unknown.
Rather than simply assuming some de facto objective function, 
compare them against the evidence.
This paper demonstrates how.

To select the best objective, the experiment is essentially the same except the objective is varied while the model and data are held fixed.
Now, select the objective indicating the greatest similarity between data and model.
Different objective functions have different scales, so they are normalized
such that each integrates to one, thereby representing them as probability distributions.
The normalized form of MSE is the normal distribution, for example (Hodson, 2022).
When used to evaluate model fit, the probability distribution is called a likelihood function
and its output the likelihood.
So, to select among objectives, compare their likelihoods, and favor the most likely.
Taking the natural logarithm of the likelihood, denoted as $\ell$,
does not change the models ranks
but simplifies the math by converting products to sums:
likelihoods multiply, so log likelihoods add.


So, to benchmark objectives, compare their log likelihoods.
The maximum likelihood estimators for the log likelihoods of several objectives are implemented below.

In [2]:
# compute likelihood
import numpy as np
from scipy.stats import pearsonr
import scipy.stats


def normal_ll(y, y_hat, transform=None, gradient=1):
    '''Log likelihood for the normal distribution with change of variable
    
    The normal distribution is the formal likelihood for the mean squared error (MSE).
    

    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.
        
    Proof
    -----
    https://www.statlect.com/probability-distributions/normal-distribution
    '''
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)
        
    e = y - y_hat
    n = len(e)
    sigma = np.sqrt(np.var(e))
    #sigma = e.std()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = -n * np.log(sigma) - n/2*np.log(2*np.pi) - 1/(2*sigma**2) * (e**2).sum() + log_gradient
    return ll


def laplace_ll(y, y_hat, transform=None, gradient=1):
    '''Log likelihood for Laplace distribution with change of variable
    
    The laplace distribution is the formal likelihood for the mean absolute
    error (MAE).
    
    Parameters
    ----------
    y : array_like
        Observations.
    y_hat : array_like
        Predictions.
    transform : function
        Change of variable transformation.
    gradient : function
        Gradient of the transform function.
    '''
    if transform is not None:
        y = transform(y)
        y_hat = transform(y_hat)
        
    e = (y - y_hat).abs()
    n = len(e)
    b = e.mean()
    log_gradient = np.sum(np.log(np.abs(gradient)))
    ll = -n * np.log(2*b) - 1/b * e.sum() + log_gradient
    return ll.sum()
                                   

def msre_ll(y, y_hat):
    '''Log likelihood for mean squared square-root error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return normal_ll(y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1/(2*np.sqrt(y)))


def mare_ll(y, y_hat):
    '''Log likelihood for mean absolute square-root error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return laplace_ll(y, y_hat, transform=lambda x: np.sqrt(x), gradient=-1/(2*np.sqrt(y)))


def lognormal_ll(y, y_hat):
    '''Lognormal log likelihood
    
    The lognormal distribution is the formal likelihood for the mean squared
    log error (MSLE).
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return normal_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1/y)


def mspe_ll(y, y_hat):
    '''Log likelhood for mean squared percentage error
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    
    '''
    return normal_ll(y, y_hat, transform=lambda x: x/y, gradient=-1/(y**2)) 


def nse_ll(y, y_hat, group='gage_id'):
    '''Log likelihood for normalized squared error (NSE)
    
    NSE is equivalent to the Nash–Sutcliffe model efficiency coefficient.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    sigma_o = y.groupby('gage_id').transform(lambda x: x.std())
    return normal_ll(y, y_hat, transform=lambda x: x/sigma_o, gradient=1/sigma_o)


def loglaplace_ll(y, y_hat):
    '''Log likelihood for log Laplace distribution
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return laplace_ll(y, y_hat, transform=lambda x: np.log(x), gradient=1/y)


def uniform_ll(y, y_hat):
    '''Log likelihood for uniform distribution.
    
    The uniform log likelihood minimizes the maximum error.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    e = np.abs(y - y_hat)
    n = len(e)
    #ll = -n * np.log(e.max()-e.min()) # standard formulation
    ll = -n * np.log(e.max() - 0)
    return ll


def bernoulli_ll(y, y_hat, groupby=None):
    '''TODO and use within zi_ll
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    pass



def zi_ll(y, y_hat, ll=normal_ll, threshold=0.01, groupby=None):
    ''' Zero-inflated log likelihood.
    
     Parameters
    ----------
    y : array_like
    y_hat : array_like
    ll : function
        Zero-inflated log likelihood 
    threshold : float
        Value below which is treated as zero
    groupby : string
        Optional groupby term (testing)
    '''
    y_o = y <= threshold
    y_hat_o = y_hat <= threshold
    
    if groupby is None:
        n1 = (y_o & y_hat_o).sum() # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).sum() # incorrect zero-flow prediction 
    else:
        n1 = (y_o & y_hat_o).groupby(groupby).sum() # correct zero-flow prediction
        n2 = (y_o ^ y_hat_o).groupby(groupby).sum() # incorrect zero-flow prediction

    n3 = (~y_o & ~y_hat_o) # correct flow predictions
    
    # fraction of correctly predicted zero flows
    rho = np.where( (n1+n2) == 0, 0, n1 / (n1 + n2))
    n_rho = 1-rho
    
    # n1 * np.log(rho) + n2 * np.log(1-rho)
    ll_zero = n1[rho!=0] * np.log(rho[rho!=0]) + n2[n_rho!=0]* np.log(n_rho[n_rho!=0])
    
    return ll_zero.sum() + ll(y[n3], y_hat[n3])


def zilognormal_ll(y, y_hat):
    '''Log likelihood for zero-inflated lognormal.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
       
    return zi_ll(y, y_hat, ll=lognormal_ll, threshold=0.01)


def ziloglaplace_ll(y, y_hat):
    '''Log likelihood for zero-inflated laplace.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    return zi_ll(y, y_hat, ll=loglaplace_ll, threshold=0.01)


## Weights
Given a set of $m$ models,
the "weight" of evidence for each model $w_i$ is
$$
w_i = \frac{ x^{-\hat H_i} }{ \sum^{m}_{i} x^{-\hat H_i}  }
$$
where the base $x$ is 2 for bits or $e$ for nats (adapted from Burnham and Anderson, 2002).

In [3]:
def compute_weights(series, base=np.e):
    '''Compute posterior weights
    
    Parameters
    ----------
    series : array_like
        Log likelihoods
    base: float
        Base of the logarithm used to compute log likelihood
    '''
    s = base**series
    return s/s.sum()

## Benchmark demonstration
To demonstrate, we benchmark the entropies of several objective functions that might be considered for a streamflow model.
The test data are streamflow observations from 1,385 streamgages in the conterminous United States \citep{Russell_2020};
roughly 14 million observations.
As streamflow can be zero or negative, which is undefined for some objective functions,
flows below 0.0028 m$^3$ s$^{-1}$ (0.01 ft$^3$ s$^{-1}$) were thresholded and treated as the ``zero-flow'' state in the comparison.
Different thresholds may yield slightly different results,
particularly among logged objectives because of their greater sensitivity near zero.

The model is simple: 
predict streamflow at a location by scaling the nearest concurrent observation by the ratio of the two drainage areas.
So when predicting flow in a large river using observations from a smaller one, scale up the observations accordingly.
By nature, the predictions are out of sample, so neither cross validation nor bias adjustment is necessary. 
We chose this example because it represents the case
in which the model is physically correct, but its boundary conditions are uncertain (a common problem in Earth science).
An alternate experiment takes a general model, 
like a neural network or a physical simulation,
then calibrates it to each objective.
The former experiment tests how well the objectives represent uncertainty about the model input,
whereas the latter tests how well they represent uncertainty about the model structure as well.

In [4]:
# load data from s3 (run once)
import pandas as pd
import numpy as np
import fsspec

fs_read = fsspec.filesystem('s3', anon=True, client_kwargs={'endpoint_url': "https://renc.osn.xsede.org"})

with fs_read.open('s3://rsignellbucket2/hytest/thodson/gages2_nndar.parquet') as f:
    df = pd.read_parquet(f)
    
# save local copy
df.to_parquet('gages2_nndar.parquet')

KeyboardInterrupt: 

In [5]:
# read local copy
import pandas as pd
import numpy as np

df = pd.read_parquet('gages2_nndar.parquet')
df[df < 0.01] = 0.01

In [6]:
# step 1: create a table of objective functions
objectives = {
    'U' : {'name':'uniformly distributed error', 'f':uniform_ll},
    'MSE' : {'name':'mean squared error', 'f':normal_ll},
    'NSE' : {'name':'normalized squared error', 'f':nse_ll},
    'MAE' : {'name': 'mean absolute error', 'f':laplace_ll},
    'MSPE' : {'name': 'mean squared percent error', 'f':mspe_ll},
    'MSLE' : {'name':'mean squared log error*', 'f':lognormal_ll},
    'MALE' : {'name':'mean absolute log error*', 'f':loglaplace_ll},
    'ZMSLE' : {'name':'zero-inflated MSLE', 'f':zilognormal_ll},
    'ZMALE' : {'name':'zero-inflated MALE', 'f':ziloglaplace_ll},
    'MARE' : {'name':'mean absolute square root error', 'f':mare_ll},
}

obj_df = pd.DataFrame.from_dict(objectives, orient='index')

# step 2: compute the information in each objective function
for index, row in obj_df.iterrows():
    # nats is the negative log likelihood or the info in the error
    obj_df.loc[index, 'bits'] = - row.f(df.obs, df.NNDAR)/len(df)/np.log(2)

# step 3: compute weights
obj_df['weight'] = compute_weights(-obj_df.bits, base=2)

# step 4: format output table

table = obj_df[['name','bits','weight']].sort_values('weight').round(2)#.rename(columns=names)

table['rank'] = len(table) - np.argsort(table['weight'])

In [7]:
obj_df

,name,f,bits,weight
U,uniformly distributed error,<function uniform_ll at 0x7f429856e480>,18.172241,0.000093
MSE,mean squared error,<function normal_ll at 0x7f429856eb60>,11.616919,0.008716
NSE,normalized squared error,<function nse_ll at 0x7f429856e340>,11.195196,0.011675
MAE,mean absolute error,<function laplace_ll at 0x7f429856e980>,9.492831,0.037994
MSPE,mean squared percent error,<function mspe_ll at 0x7f429856e2a0>,23.537709,0.000002
MSLE,mean squared log error*,<function lognormal_ll at 0x7f429856e0c0>,7.466534,0.154772
MALE,mean absolute log error*,<function loglaplace_ll at 0x7f429856e3e0>,7.039863,0.208034
ZMSLE,zero-inflated MSLE,<function zilognormal_ll at 0x7f42e404cc20>,7.182802,0.188411
ZMALE,zero-inflated MALE,<function ziloglaplace_ll at 0x7f429856e660>,6.952254,0.221059
MARE,mean absolute square root error,<function mare_ll at 0x7f425d408040>,7.337572,0.169245


Results are shown in the table below.
In the experiment, the data and model were fixed;
therefore, so was the information in the error.
All that changed was how we measured it.
Relative to ZMALE, the excess bits in the other objective functions are noise.
So, MSE measures at least 40 percent noise,
and NSE at least 38 percent.
Consider stochastic gradient descent, where noisier gradients require more iterations to reach the solution.
In that case, each iteration completes faster, so the solution may be reached quicker overall.
A poorly chosen objective incurs a similar penalty but potentially without benefit.
In general, noisier objectives convey less information and so require more iterations during calibration,
more data to reach the solution,
and produce models that require more storage space
(better model, better data compression).

In [8]:
#print(table.to_latex())
table

,name,bits,weight,rank
MSPE,mean squared percent error,23.54,0.00,10
U,uniformly distributed error,18.17,0.00,9
MSE,mean squared error,11.62,0.01,8
NSE,normalized squared error,11.20,0.01,7
MAE,mean absolute error,9.49,0.04,6
MSLE,mean squared log error*,7.47,0.15,5
MARE,mean absolute square root error,7.34,0.17,4
ZMSLE,zero-inflated MSLE,7.18,0.19,3
MALE,mean absolute log error*,7.04,0.21,2
ZMALE,zero-inflated MALE,6.95,0.22,1


### Bootstrap

In [9]:
sample=100
n = 1000
b = 100
seed=12345
block_size = n//b

In [10]:
def nse_ll(y, y_hat, sigma_o=None, group='gage_id'):
    '''Log likelihood for normalized squared error (NSE)
    
    NSE is equivalent to the Nash–Sutcliffe model efficiency coefficient.
    
    Parameters
    ----------
    y : array_like
    y_hat : array_like
    '''
    if sigma_o is None:
        sigma_o = y.groupby('gage_id').transform(lambda x: x.std(ddof=0))
        
    return normal_ll(y, y_hat, transform=lambda x: x/sigma_o, gradient=1/sigma_o)


def nse2_ll(y, y_hat):
    return nse_ll(y, y_hat, sigma_o=sigma_global)
    

In [11]:
#modify nse for the bootstrap
objectives = {
    'U' : {'name':'uniformly distributed error', 'f':uniform_ll},
    'MSE' : {'name':'mean squared error', 'f':normal_ll},
    'NSE' : {'name':'normalized squared error', 'f':nse2_ll},
    'MAE' : {'name': 'mean absolute error', 'f':laplace_ll},
    'MSPE' : {'name': 'mean squared percent error', 'f':mspe_ll},
    'MSLE' : {'name':'mean squared log error*', 'f':lognormal_ll},
    'MALE' : {'name':'mean absolute log error*', 'f':loglaplace_ll},
    'ZMSLE' : {'name':'zero-inflated MSLE', 'f':zilognormal_ll},
    'ZMALE' : {'name':'zero-inflated MALE', 'f':ziloglaplace_ll},
    'MARE' : {'name':'mean absolute square root error', 'f':mare_ll},
}

obj_df = pd.DataFrame.from_dict(objectives, orient='index')

## Run Bootstrap

In [ ]:

sigma_o = df['obs'].groupby('gage_id').transform(lambda x: x.std())

boot_array = np.zeros((block_size, obj_df.f.shape[0]))

for b in range(0,b): # for each block
    for i in range(0,block_size):
        state = b*block_size + i
        #temp_df = df.sample(df.shape[0], replace=True, random_state=state)
        temp_df = df.groupby('gage_id').sample(sample, replace=True, random_state=state)
        sigma_global = sigma_o.groupby('gage_id').head(sample)
        
        for j,f in enumerate(obj_df.f):
            #print(f)
            boot_array[i,j]= f(temp_df.obs, temp_df.NNDAR)/len(temp_df)/np.log(2)
    
    boot_df = pd.DataFrame(boot_array)
    boot_df.columns = obj_df.index
    boot_df.to_csv('bootstrap/h_boot_{}.csv'.format(b))

## read in bootstraps

In [11]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = 'bootstrap'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True).iloc[:,1:]

#big_df.columns = obj_df.index

In [12]:
cols = table.index
vars = big_df.var()

In [13]:
vars[cols]

MSPE     0.553174
U        0.374864
MSE      0.022104
NSE      2.053865
MAE      0.000423
MSLE     0.000049
MARE     0.000050
ZMSLE    0.000043
MALE     0.000041
ZMALE    0.000035
dtype: float64

## Data availability
The streamflow data are from Russell et al. (2020) and are available at https://doi.org/10.5066/P9XT4WSP.
This demonstration notebook is available at at https://code.usgs.gov/wma/hytest/workflow-hodson-2022-objective-benchmark. 


## References 
Burnham, K.P. and Anderson, D.R. (2002). Model selection and multimodel inference: A Practical Information-Theoretic Approach. 2nd Edition, Springer-Verlag, New York.

Hodson, T.O. (2022). Root-mean-square error (RMSE) or mean absolute error (MAE): when to use them or not, Geosci. Model Dev., 15, 5481–5487. https://doi.org/10.5194/gmd-15-5481-2022

Russell, A.M., Over, T.M., and Farmer, W.H. (2020). Cross-validation results for five statistical methods of daily streamflow estimation at 1,385 reference streamgages in the conterminous United States, Water Years 1981-2017: U.S. Geological Survey data release. https://doi.org/10.5066/P9XT4WSP

In [14]:
df

obs    NNDAR
gage_id         date                      
01013500        1980-10-01  509.0   649.55
                1980-10-02  518.0   618.91
                1980-10-03  516.0   618.91
                1980-10-04  620.0   796.61
                1980-10-05  759.0  1397.14
...                           ...      ...
402114105350101 2017-09-26   23.4    23.32
                2017-09-27   22.1    22.79
                2017-09-28   23.8    26.72
                2017-09-29   28.4    26.99
                2017-09-30   25.5    25.15

[14067063 rows x 2 columns]